# Supervised

This notebook extracts respiration signals, for each subject in the dataset, with the following models:
- MTTS-CAN
- BigSmall

In [10]:
import os
import numpy as np
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])
scenarios

[('Proband01', '101_natural_lighting'),
 ('Proband02', '101_natural_lighting'),
 ('Proband03', '101_natural_lighting'),
 ('Proband04', '101_natural_lighting'),
 ('Proband05', '101_natural_lighting'),
 ('Proband06', '101_natural_lighting'),
 ('Proband07', '101_natural_lighting'),
 ('Proband08', '101_natural_lighting'),
 ('Proband09', '101_natural_lighting'),
 ('Proband10', '101_natural_lighting'),
 ('Proband11', '101_natural_lighting'),
 ('Proband12', '101_natural_lighting'),
 ('Proband13', '101_natural_lighting'),
 ('Proband14', '101_natural_lighting'),
 ('Proband15', '101_natural_lighting'),
 ('Proband16', '101_natural_lighting'),
 ('Proband17', '101_natural_lighting'),
 ('Proband18', '101_natural_lighting'),
 ('Proband19', '101_natural_lighting'),
 ('Proband20', '101_natural_lighting'),
 ('Proband21', '101_natural_lighting'),
 ('Proband22', '101_natural_lighting'),
 ('Proband23', '101_natural_lighting'),
 ('Proband24', '101_natural_lighting'),
 ('Proband25', '101_natural_lighting'),


## Conduct experiments

In [11]:
from datetime import datetime

manifest = {
    'timestamp_start': datetime.now(),
    'scenarios': scenarios,
}

In [12]:
import respiration.utils as utils
import respiration.extractor.mtts_can as mtts_can
import respiration.extractor.big_small as big_small

device = utils.get_torch_device()

frame_depth = 10
mtts_model = mtts_can.load_model(frame_depth=frame_depth)
big_small_model, _ = big_small.load_model(device=device)

extracted_signals = []

# TODO: Use batch processing
for (subject, setting) in scenarios:
    print(f'Processing {subject} - {setting}')

    frames, meta = dataset.get_video_rgb(subject, setting)

    #
    # MTTS-CAN model to extract the signal
    #

    mtts_start = datetime.now()

    resized, normalized = mtts_can.preprocess_video_frames(frames)
    cutoff = mtts_can.calculate_cutoff(resized.shape[0], frame_depth)

    resized = resized[:cutoff]
    normalized = normalized[:cutoff]

    _, respiration = mtts_model.predict(
        (resized, normalized),
        batch_size=100
    )
    extracted_signals.append({
        'subject': subject,
        'setting': setting,
        'model': 'mtts_can',
        'duration': datetime.now() - mtts_start,
        'sampling_rate': meta.fps,
        'signal': respiration.squeeze().tolist(),
    })

    #
    # Use the BigSmall model to extract the signal
    #

    big_small_start = datetime.now()
    big_small_raw = big_small.batch_process(big_small_model, frames, device)
    extracted_signals.append({
        'subject': subject,
        'setting': setting,
        'model': 'big_small',
        'duration': datetime.now() - big_small_start,
        'sampling_rate': meta.fps,
        'signal': big_small_raw.tolist(),
    })

    # Garbage collect the frames
    del frames

Processing Proband01 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
Processing Proband02 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Processing Proband03 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Processing Proband04 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Processing Proband05 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Processing Proband06 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Processing Proband07 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Processing Proband08 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Processing Proband09 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Processing Proband10 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Processing Proband11 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Processing Proband12 - 101_natural_lighting
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/ste

In [13]:
manifest['timestamp_finish'] = datetime.now()

In [14]:
import pandas as pd

df = pd.DataFrame(extracted_signals)
df.head()

,subject,setting,model,duration,sampling_rate,signal
0,Proband01,101_natural_lighting,mtts_can,0 days 00:00:03.832123,30,"[0.5543686151504517, 0.4143042266368866, 0.193..."
1,Proband01,101_natural_lighting,big_small,0 days 00:00:21.100647,30,"[-0.06490246951580048, -0.0634026825428009, -0..."
2,Proband02,101_natural_lighting,mtts_can,0 days 00:00:03.012597,30,"[0.27873915433883667, -0.143912672996521, 0.35..."
3,Proband02,101_natural_lighting,big_small,0 days 00:00:18.561170,30,"[-0.6646443009376526, -0.7338840365409851, -0...."
4,Proband03,101_natural_lighting,mtts_can,0 days 00:00:02.981912,30,"[-0.07861271500587463, -0.34626856446266174, -..."


In [15]:
output_dir = utils.dir_path('outputs', 'signals', mkdir=True)

# Save the evaluation dataframe
csv_path = os.path.join(output_dir, 'pretrained_predictions.csv')
df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(output_dir, 'pretrained_manifest.json')
utils.write_json(json_path, manifest)